In [1]:
pip install -U transformers memory_profiler datasets

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/3e/6b/1b589f7b69aaea8193cf5bc91cf97410284aecd97b6312cdb08baedbdffe/transformers-4.38.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/131.1 kB ? eta -:--:--
     --- ------------------------------------ 10.2/131.1 kB ? eta -:--:--
     ----------- ------------------------- 41.0/131.1 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 131.1/131.1 kB 1.9 MB/s eta 0:00:00
  Obtaining dependency information for memory_profiler from https://files.pythonhosted.org/packages/49/26/aaca612a0634ceede20682e692a6c55e35a94c21ba36b807cc40fe910ae1/memory_profiler-0.61.0-py3-none-any.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/73/75/ead8e4489217835fd8004c9c89dad8217deacd7e2591340dbe249b58c29f/datasets-2.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for filelock from https://file

In [4]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7b/0e/25d6b5678ed3c7e12bc94d047d0e9492e89cc78b7ea0034ac0f1cf2ff304/scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/9a/25/5b30cb3efc9566f0ebeaeca1976150316353c17031ad7868ef46de5ab8dc/scipy-1.12.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 643.6 kB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2

In [5]:
import torch, gc, random, datasets
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [6]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)

In [7]:
df = pd.read_csv("./like_data.csv")

In [8]:
df

,posts,likes
0,That’s my truck and there’s 51 boxes on it. Yo...,83
1,Righteous beat down of the poser Knicks by the...,5
2,I think the devs have said Bluesky has a decen...,57
3,Stunning and brave of you,2
4,I saw down with the Grand Wizard for some coff...,162
5,this is me if you even care,30
6,sketch of my living room,22
7,They're adding online icons 🟢 to Bluesky to sh...,75
8,バビロニア,1707
9,We just passed 100k users in a day! \n\nWhen w...,2830


In [9]:
# print("Range:"+"("+str(df.total_score.min())+", "+str(df.total_score.max())+")")

In [10]:
# df['Review Text'].str.split(' ').str.len().max()

In [11]:
# df = df[~df.total_score.isna()]
# # df = df[~(df.score_1.isin(['0,0', '0,5', '1,0', '1,5', '2,0',]))]
# df = df[~(df.total_score.astype(float)==0)]

In [12]:
# len(df)

In [13]:
def LinearRegression_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int):

  # Make data
  X = Data
  y = Target


  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Call the Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Encode the text
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          item["labels"] = float(item["labels"])
          return item

      def __len__(self):
          return len(self.labels)

  # convert our tokenized data into a torch Dataset
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Call Model
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda") # np.log(1000)

  # def inverse_sigmoid(logits):
  #   return -np.log((1.0 / (logits + 1e-8)) - 1.0)

  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import r2_score

  def compute_metrics_for_regression(eval_pred):
      logits, labels = eval_pred
      # print("Logits:", logits[0:5])
      # print("Labels:", labels[0:5])
      # logits = inverse_sigmoid(logits)
      labels = labels.reshape(-1, 1)
      
      print("Logits:", logits[0:5])
      print("Labels:", labels[0:5])
      # print("Labels:", labels)

      mse = mean_squared_error(labels, logits)
      rmse = mean_squared_error(labels, logits, squared=False)
      mae = mean_absolute_error(labels, logits)
      r2 = r2_score(labels, logits)
      # smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
      single_squared_errors = ((logits - labels).flatten()**2).tolist()
      accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
      
      return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}

  # Specifiy the arguments for the trainer  
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=64,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      weight_decay=0.01,               # strength of weight decay
      learning_rate=2e-5,
      # logging_dir='./logs',            # directory for storing logs
      save_total_limit=10,
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = 'accuracy',    # select the base metrics
      evaluation_strategy="epoch",
      save_strategy="epoch",
  ) 

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics_for_regression,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()
  
  # Call the summary
  trainer.evaluate()



  return trainer, model

In [ ]:
%%time
%%memit
bert_trainer, bert_model = LinearRegression_with_Transformer(model_name = 'google-bert/bert-base-multilingual-cased', 
                                                                 Data = df.posts.astype(str), 
                                                                 Target = df.likes.astype(float), 
                                                                 test_size = 0.2, 
                                                                 max_length = 512, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 100)

In [16]:
%%time
%%memit
bert_trainer, bert_model = LinearRegression_with_Transformer(model_name = 'Twitter/twhin-bert-base', 
                                                                 Data = df.posts.astype(str), 
                                                                 Target = df.likes.astype(float), 
                                                                 test_size = 0.2, 
                                                                 max_length = 512, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 100)

tokenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<00:00, 46.0kB/s]
c:\Users\samuel\bluesky\myenv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samuel\.cache\huggingface\hub\models--google-bert--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|█

AssertionError: Torch not compiled with CUDA enabled

In [21]:
df2 = df[3000:].copy()
df3 = df[0:3000].copy()

In [24]:
%%time
%%memit
sbert_trainer, sbert_model = LinearRegression_with_Transformer(model_name = 'bert-base-uncased', 
                                                                 Data = df2['Review Text'].astype(str), 
                                                                 Target = df2.Rating.astype(float), 
                                                                 test_size = 0.66, 
                                                                 max_length = 126, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 5)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Accuracy
1,No log,0.507760,0.507760,0.712573,0.491016,0.581568,0.653206
2,No log,0.434158,0.434158,0.658906,0.438609,0.642221,0.662969
3,No log,0.469371,0.469371,0.685106,0.449530,0.613203,0.669477
4,No log,0.440199,0.440199,0.663475,0.432040,0.637242,0.668738
5,0.769400,0.447230,0.447231,0.668753,0.439436,0.631448,0.669107


***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[3.3508034]
 [4.6908293]
 [4.8653245]
 [4.8855553]
 [4.9706054]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-109
Configuration saved in ./results/checkpoint-109/config.json
Model weights saved in ./results/checkpoint-109/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-246] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


...

Logits: [[3.8679595]
 [4.954284 ]
 [5.03974  ]
 [5.0272856]
 [4.743147 ]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-436
Configuration saved in ./results/checkpoint-436/config.json
Model weights saved in ./results/checkpoint-436/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-984] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[3.782437 ]
 [4.971796 ]
 [5.0880117]
 [5.059329 ]
 [4.7667904]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-545
Configuration saved in ./results/checkpoint-545/config.json
Model weights saved in ./results/checkpoint-545/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1230] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-327 (score: 0.6694771096812366).
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[4.0661874]
 [5.004098 ]
 [5.0948296]
 [5.0855093]
 [4.8016233]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]
peak memory: 5813.04 MiB, increment: 0.68 MiB
CPU times: user 12min 42s, sys: 15.6 s, total: 12min 57s
Wall time: 12min 47s


In [ ]:
class MakeTorchData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
test_embeddings = tokenizer(df3['Review Text'].astype(str).tolist(), truncation=True, padding=True, max_length=126)
test_dataset = MakeTorchData(test_embeddings, df3.Rating.astype(float))

In [36]:
sbert_trainer.eval_dataset = test_dataset
sbert_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3000
  Batch size = 20


Logits: [[5.065415 ]
 [5.135058 ]
 [3.0691428]
 [5.123178 ]
 [5.1684527]]
Labels: [[4.]
 [5.]
 [3.]
 [5.]
 [5.]]


{'epoch': 5.0,
 'eval_accuracy': 0.6603333333333333,
 'eval_loss': 0.48832669854164124,
 'eval_mae': 0.45958763360977173,
 'eval_mse': 0.4883267283439636,
 'eval_r2': 0.6103774787626888,
 'eval_rmse': 0.6988037824630737,
 'eval_runtime': 13.5094,
 'eval_samples_per_second': 222.068,
 'eval_steps_per_second': 11.103}